In [ ]:
import pypdf
import sys
import warnings
import openai
import pandas as pd
import datetime
from datetime import timedelta
import openai
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
# this key is already expired
openai.api_key = 'sk-proj-m1A1Xu1cfrX1i1gOB54PT3BlbkFJlElhAJmxCyI5klbOJp4h'

## 1.Obtain the data set required by the neural network (embeddings_num_df)

In [1]:
def readDoc(pdfPATH):
    pdfFileObj = open(pdfPATH, 'rb')
    pdfReader = pypdf.PdfReader(pdfFileObj)
    page_num = len(pdfReader.pages)
    document = ""
    for i in range(page_num):
        pageObj = pdfReader.pages[i]
        document += pageObj.extract_text()
    pdfFileObj.close()
    
    document = document.replace('\n', '')
    document = document.replace('\uf0b7', '')
    document.strip()

    start_marker = "What happened"
    end_marker = "What I’ve decided – and why"
    start_index = document.find(start_marker)
    end_index = document.find(end_marker)

    if start_index != -1 and end_index != -1 and start_index < end_index:
        document = document[start_index + len(start_marker):end_index]
    else:
        document = ""
    return document

In [ ]:
documents_list = []
decisions_list = []
!rm ./metadata.csv
!rm -rf ./decisions
for i in range(5):
    print(f'iteration {i}')
    current_date = datetime.datetime.strptime("2024-07-08", "%Y-%m-%d")
    date = current_date - timedelta(days=i*90)
    date = str(date).replace(' 00:00:00', '').strip()
    print(date)
    !python scrape.py get-metadata --industry-sector "payment-protection-insurance,insurance" --to {date}
    if not os.path.isfile('./metadata.csv'):
        print('no files')
        continue
    %run scrape.py download-decisions
    df_meta = pd.read_csv('./metadata.csv')
    pdf_folder_path = './decisions'
    
    for idx,file_name in enumerate(os.listdir(pdf_folder_path)):
        document = readDoc(pdf_folder_path + '/' + file_name)
        documents_list.append(document)
        decision = df_meta['decision'].iloc[idx]
        decisions_list.append(decision)
    !rm ./metadata.csv
    !rm -rf ./decisions


model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(documents_list)
embeddings_df = pd.DataFrame(embeddings)
labels_df = pd.DataFrame({'label': decisions_list})
embeddings_num_df = pd.concat([embeddings_df, labels_df], axis=1)
# embeddings_num_df.to_csv('embeddings_num_df.csv', index=False)

## 2. Extract the information with the API of GPT-3.5-turbo(data_all)

In [ ]:
def get_insurance_info(input_text):
    INPUT_PROMPT = f"""
    Please analyze the given text and provide the following information. Ensure that the answers strictly adhere to the options given below, and the results contain only words:
    1. type of insurance
    2. is the premium/payment issue? 
    3. is the service attitude/communication issue?
    4. is the fail to meet customer's expectations issue?
    5. is the claims processing delays issue?
    6. is the policy terms issue?
    7. gender

    The answer options for each question are as follows, and you can not choose answers outside the parentheses:
    1. type of insurance:(Animal、Legal Expenses and Monetary Loss、Health、Household and Commercial Property、Travel、Motor、Personal Belongings、Payment Protection、Others)
    2. is the premium/payment issue:(Yes/No)
    3. is the service attitude/communication issue:(Yes/No)
    4. is the fail to meet customer's expectations issue:(Yes/No)
    5. is the claims processing delays issue:(Yes/No)
    6. is the policy terms issue:(Yes/No)
    7. gender:(Male/Female/Male and Female)

    The output should be in the following format:
    type of insurance: Your answer here
    premium/payment issue: Your answer here
    service attitude/communication issue: Your answer here
    fail to meet customer's expectations issue: Your answer here
    claims processing delays: Your answer here
    policy terms issues: Your answer here
    gender: Your answer here

    Text here: {input_text}
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": INPUT_PROMPT}
        ],
        max_tokens=1500, 
        temperature=0,  
        n=1,  
        stop=["###"]  
    )

    response_text = response.choices[0].message["content"].strip()
    answers = response_text.split('\n')
    features = {
        "type of insurance": answers[0].split(":")[1].strip() if len(answers) == 7 else 'None',
        "premium/payment issue": answers[1].split(":")[1].strip() if len(answers) == 7 else 'None',
        "service attitude/communication issue": answers[2].split(":")[1].strip() if len(answers) == 7 else 'None',
        "fail to meet customer's expectations issue": answers[3].split(":")[1].strip() if len(answers) == 7 else 'None',
        "claims processing delays": answers[4].split(":")[1].strip() if len(answers) == 7 else 'None',
        "policy terms issues": answers[5].split(":")[1].strip() if len(answers) == 7 else 'None',
        "gender": answers[6].split(":")[1].strip() if len(answers) == 7 else 'None'

    }
    
    return features

In [ ]:
df_all = pd.DataFrame()
!rm -rf ./decisions
!rm -rf ./metadata.csv
for i in range(5):
    print(f'iteration {i}')
    df = pd.DataFrame(columns=["decision_id",
                               "type of insurance",
                               "premium/payment issue",
                               "service attitude/communication issue",
                               "fail to meet customer's expectations issue",
                               "claims processing delays",
                               "policy terms issues",
                               "gender"])

    current_date = datetime.datetime.strptime("2024-07-08", "%Y-%m-%d")
    date = current_date - timedelta(days=i*90)
    date = str(date).replace(' 00:00:00', '').strip()
    print(date)
    !python scrape.py get-metadata --industry-sector "payment-protection-insurance,insurance" --to {date}
    if not os.path.isfile('./metadata.csv'):
        print('no files')
        continue
    %run scrape.py download-decisions
    
    df_meta = pd.read_csv('./metadata.csv')

    pdf_folder_path = './decisions'
    documents_list = []
    name_list = []
    for file_name in os.listdir(pdf_folder_path):
        document = readDoc(pdf_folder_path + '/' + file_name)
        documents_list.append(document)
        name_list.append(file_name.replace('.pdf', '').strip())
        
    for i in range(len(documents_list)):
        features = get_insurance_info(documents_list[i])
        features["decision_id"] = name_list[i]
        df = pd.concat([df, pd.DataFrame([features])], ignore_index=True)
    
    df = pd.merge(df, df_meta, on="decision_id", how='left')
    df_all = pd.concat([df_all, df], ignore_index=True)
    
    # df_all.to_csv('data_all.csv')
    !rm ./metadata.csv
    !rm -rf ./decisions